In [1]:
import pandas as pd

In [47]:
toxcast_genes = pd.read_csv("Assay_Summary_151020.csv",encoding="latin1")

In [48]:
toxcast_genes = pd.DataFrame([(r.aeid,y) for i,r in toxcast_genes[toxcast_genes.intended_target_entrez_gene_id.isnull()==False
                             ][["aeid","intended_target_entrez_gene_id"]].iterrows() 
                              for y in r.intended_target_entrez_gene_id.split("|")],columns=["aeid","gene_id"])

In [52]:
cooccurs = pd.read_table("cooccurs.txt")

In [53]:
cooccurs

,uid1,uid2,uid1_name,uid2_name,npmi
0,D000001,D000067470,Calcimycin,Cellular Reprogramming Techniques,0.062200
1,D000001,D000067956,Calcimycin,Adenylyl Cyclase Inhibitors,0.049381
2,D000001,D000068438,Calcimycin,Brimonidine Tartrate,0.068360
3,D000001,D000071425,Calcimycin,"Src Homology 2 Domain-Containing, Transforming...",0.009244
4,D000001,D000071876,Calcimycin,Prostaglandin-E Synthases,0.014464
5,D000001,D000072000,Calcimycin,NF-KappaB Inhibitor alpha,0.002573
6,D000001,D000072197,Calcimycin,ADAM10 Protein,0.032844
7,D000001,D000072216,Calcimycin,Zona Pellucida Glycoproteins,0.078340
8,D000001,D000072377,Calcimycin,Syk Kinase,0.038342
9,D000001,D000072501,Calcimycin,Cytochrome P450 Family 27,0.009041


In [ ]:
seed_terms = {
    "Angiogenesis": {
        "Neovascularization, Pathologic": {"uid":"D009389"},
        "Neovascularization, Physiologic":{"uid":"D018919"}}, 
    "Apoptosis":{
        "Apoptosis": {"uid":"D017209"}},
    "Cell cycle":{
        "Cell cycle":{"uid":"D002453"}},
    "Epigenetics": {
        "Epigenomics": {"uid":"D057890"}},
    "Genotoxicity":{
        "DNA damage": {"uid":"D004249"},
        "DNA repair": {"uid":"D004260"}},
    "Growth hormones":{
        "Growth hormone": {"uid":"D013006"}},
    "Immortalization": {
        "Cell survival": {"uid":"D002470"}},
    "Immunomodulation": {
        "Immune system":{"uid":"D007107"}},
    "Inflammation":{
        "Inflammation": {"uid":"D007249"}},
    "Oxidative stress": {
        "Oxidative stress": {"uid":"D018384"}},
    "Proliferation":{
        "Cell proliferation": {"uid":"D049109"}},
    "Steroid hormones":{
        "Gonadal steroid hormones": {"uid":"D012739"}},
    "Xenobiotic metabolism": {
        "Xenobiotics": {"uid":"D015262"}}
}

In [ ]:
for cc,mt_dict in seed_terms.items():
    for mt_name,dd in mt_dict.items():
        uid = dd["uid"]
        subset = cooccurs[(cooccurs.uid1==uid)|(cooccurs.uid2==uid)].copy()
        search_terms = set(subset[(subset.ct>2)&(subset.npmi>subset[((subset.uid1.isin(limit_uids))|(subset.uid2.isin(limit_uids)))
               &(subset.ct>2)&(subset.npmi>0.0)].sort_values("npmi",ascending=False).npmi.quantile(0.8))&
             ((subset.uid1.isin(limit_uids))|(subset.uid2.isin(limit_uids)))].sort_values("npmi",ascending=False).apply(
            lambda x: x.uid1 if x.uid1!=uid else x.uid2,axis=1).values)
        search_terms.add(uid)
        cols = []
        for t in search_terms:
            a = emcon[emcon.uid==t]
            cutoff = a.npmi.quantile(0.9)
            cols.append(a[a.npmi>cutoff])
        genes = pd.concat(cols).pivot(
            columns="uid",index="gene_id",values="npmi").rank(ascending=False).median(axis=1).sort_values()
        dd["search_terms"] = search_terms
        dd["genes"] = genes
        print(cc,mt_name,uid)

In [ ]:
for cc,mt_dict in seed_terms.items():
    data = []
    search_terms = set()
    for mt_name,dd in mt_dict.items():
        data.append(dd["genes"])
        search_terms |= set(dd["search_terms"])
    mt_dict["genes"] = pd.DataFrame(data).mean().sort_values()
    mt_dict["search_terms"] = search_terms

In [ ]:
for cc,mt_dict in seed_terms.items():
    print(cc,len(mt_dict["genes"]),len(mt_dict["search_terms"]))

In [ ]:
data = []
for co in combinations(seed_terms.keys(),2):
    data.append([co,len(set(seed_terms[co[0]]["genes"].index)),len(set(seed_terms[co[1]]["genes"].index)),len(set(seed_terms[co[0]]["genes"].index)&set(seed_terms[co[1]]["genes"].index))/len(set(seed_terms[co[0]]["genes"].index)|set(seed_terms[co[1]]["genes"].index)) ])

In [ ]:
overlap = pd.DataFrame(data,columns=["characteristics","co0 genes","co1 genes","% overlapping genes"])

In [ ]:
overlap.sort_values("% overlapping genes")